# imports

In [ ]:
import os
from PIL import Image

import torch.nn as nn
import torch
import torch.nn.functional as F
# import torch as torch
# import torch.utils.data as TData
# from torch import optim
# from torch.nn.parameter import Parameter
# from torchsummary import summary
# from torch.autograd import Variable
# import torchvision.transforms as transforms

import pandas as pd

import cv2
import json
import numpy as np
    
import matplotlib.pyplot as plt
from scipy.cluster.vq import vq, kmeans, whiten
from sklearn.cluster import KMeans

In [ ]:
# pytorch-model-summary    0.1.2       
# torch                    0.4.1       
# torchfile                0.1.0       
# torchsummary             1.5.1       
# torchtext                0.2.1       
# torchvision              0.2.0  

In [ ]:
import HeidelbergTraining.UNet.UNet_functions as unet_functions
import HeidelbergTraining.UNet.UNet_model as unet_models

In [ ]:
total_path = "./"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# loading labeled input images and matching them with their labeled segmentations

In [ ]:
normal_path = os.path.join(total_path, "data/NORMAL EYES_segmented_FIN")
diseased_path = os.path.join(total_path, "data/Segmentations diseased eyes - MacTel/MacTel eyes, no segmentation")
print(os.path.isdir(normal_path))
print(os.path.isdir(diseased_path))

In [ ]:
all_dirs = []
for dirpath, dirs, files in os.walk(normal_path):
    if len(dirs) > 0:
        all_dirs.extend(dirs)

In [ ]:
all_files = []
for i, (dirpath, dirs, files) in enumerate(os.walk(normal_path)):
    if i == 0:
        continue
    filenames = [os.path.join(normal_path, all_dirs[i - 1], f) for f in files if f.endswith(".png")]
    all_files.extend(filenames)

for i, (dirpath, dirs, files) in enumerate(os.walk(diseased_path)):
    filenames = [os.path.join(diseased_path, f) for f in files if f.endswith(".png")]
    all_files.extend(filenames)

all_image_files = [file for file in all_files if ("NHORC" in file or "MacTel eyes, no segmentation" in file)] 
for f in all_image_files:
    if not os.path.isfile(f):
        print("bruh")
    
all_image_files = list(set(all_image_files))


In [ ]:
print(len(all_image_files))

In [ ]:
normal_path = os.path.join(total_path, "data/NORMAL EYES_segmented_FIN")
diseased_path = os.path.join(total_path, "data/Segmentations diseased eyes - MacTel/MacTel eyes, segmentations indicating interruptions of  layers")
print(os.path.isdir(normal_path))
print(os.path.isdir(diseased_path))


In [ ]:
all_dirs = []
for dirpath, dirs, files in os.walk(normal_path):
    if len(dirs) > 0:
        all_dirs.extend(dirs)

In [ ]:
all_files = []
for i, (dirpath, dirs, files) in enumerate(os.walk(normal_path)):
    if i == 0:
        continue
    filenames = [os.path.join(normal_path, all_dirs[i - 1], f) for f in files if f.endswith(".png")]
    all_files.extend(filenames)

for i, (dirpath, dirs, files) in enumerate(os.walk(diseased_path)):
    filenames = [os.path.join(diseased_path, f) for f in files if f.endswith(".png")]
    all_files.extend(filenames)

    
all_mask_files = [file for file in all_files if ("NHORC" not in file and "MacTel eyes, no segmentation" not in file)]

for f in all_mask_files:
    if not os.path.isfile(f):
        print("bruh")

In [ ]:
print(len(all_mask_files))

In [ ]:
def get_numbers(file_path):
    name = file_path[file_path.find("NHORC") + 5:-4] if ("NHORC" in file_path) else file_path[file_path.find("NHOR") + 4:-4]
    words = name.split()
    new_words = []
    for w in words:
        new_words.extend(w.split(","))

    words = []
    for w in new_words:
        words.extend(w.split("_"))
        
    new_words = words
    words = []
    
    for w in new_words:
        words.extend(w.split("."))

    numbers = []
    for f in words:
        if "-" in f and f[0].isdigit():
            numbers.append(f)
            continue
        if f.isdigit():
            numbers.append(f)
    
    return numbers

# match gt file paths to image file paths

In [ ]:
image_path_list = []
mask_path_list = []

i = 0
count = 0

all_mask_files_names = [f[f.find("NHOR"):] for f in all_mask_files]

for file in sorted(all_image_files):
#     if i > 0:
# #         print(file)
#         break
        
    which_eye = "OD" if "OD" in file else "OS" if "OS" in file else ""
    
    numbers = get_numbers(file)
        
#     print(numbers)
#     print(which_eye)
    
    match = [i for i in range(len(all_mask_files_names)) if numbers[0] in all_mask_files_names[i] and numbers[1] in all_mask_files_names[i] and which_eye in all_mask_files_names[i]]
    if len(match) > 1:
        actual_match = [i for i in match if "NHORC" not in all_mask_files_names[i] and numbers[1] in all_mask_files_names[i][all_mask_files_names[i].find(numbers[0])+len(numbers[0]):]]
#         print(file)
#         print(numbers)
#         print()
#         print()
        if len(actual_match) > 0:
            mask_path_list.append("%s%s"%(count, all_mask_files[actual_match[0]]))
            image_path_list.append("%s%s"%(count, file))
            count += 1
        
        continue

    if len(match) == 1:
        actual_match = [i for i in match if "NHORC" not in all_mask_files_names[i]]
        
        if len(actual_match) > 0:
            mask_path_list.append("%s%s"%(count, all_mask_files[actual_match[0]]))
            image_path_list.append("%s%s"%(count, file))
            count += 1
#     print(match)
    
    i += 1
print(count)
image_path_list = sorted(image_path_list)
mask_path_list = sorted(mask_path_list)


# helper functions for extracting crops from OCTs

In [ ]:
def get_black_white(img):
    sums = np.sum(img, axis=0)
    
    sums = [f / 1000 for f in sums]
    diffs = [sums[i] - sums[i-1] for i in range(1,len(sums))]
    min_diffs = min(diffs)
    max_diffs = max(diffs)
    
    x_crop_start = [i for i in range(len(diffs)) if diffs[i] == min_diffs][0]
    x_crop_end = [i for i in range(len(diffs)) if diffs[i] == max_diffs][0]
    
    return img[50:-100, x_crop_start:x_crop_end], x_crop_start, x_crop_end

In [ ]:
def match_to_img(img_gt, img_to_match, pixel_move=10):
    sums1 = np.sum(img_gt, axis=1) / 10000
    sums2 = np.sum(img_to_match, axis=1) / 10000
    
    pixel_move = 10
    min_diff = np.sum(np.absolute(sums1 - sums2))
    
    moving_img = img_to_match
    
    total_pixel_move = 0
    num_iters = img_gt.shape[0] // pixel_move
    if num_iters * pixel_move < img_gt.shape[0]:
        num_iters += 1
    
    for i in range(num_iters + pixel_move):
        sums2 = np.concatenate((sums2[pixel_move:], sums2[:pixel_move]), axis=0)
        curr_diff = np.sum(np.absolute(sums1 - sums2))
        
        if curr_diff < min_diff:
            min_diff = curr_diff
            img_to_match = moving_img
            total_pixel_move = (i+1) * pixel_move
    
    return total_pixel_move

In [ ]:
def get_percentile(img, percentile):
    img = np.sort(img.ravel())
    index = (np.shape(img.ravel())[0] - 1) * percentile / 100
    return img[int(index)]

In [ ]:
def get_img_cropped(img, threshold=80):
    y_sums = np.array(np.sum(img, axis=1)) / 1000
    yaverage = int(np.mean(np.where(y_sums > np.percentile(y_sums, threshold))[0]))
    
    x_slice = img[yaverage, :]
    xaverage = int(np.mean(np.where(x_slice > np.percentile(x_slice, threshold))[0]))
    
    pad = 256
    
    ystart = yaverage - pad if yaverage - pad >= 0 else 0
    xstart = xaverage - pad if xaverage - pad >= 0 else 0
    
    yend = yaverage + pad if yaverage + pad < img.shape[0] else img.shape[0]
    xend = xaverage + pad if xaverage + pad < img.shape[1] else img.shape[1]
    
    min_length = xend - xstart if (xend - xstart) < (yend - ystart) else yend - ystart
    xaverage = xstart + (xend - xstart) // 2
    yaverage = ystart + (yend - ystart) // 2

        
    if min_length < 128:
        return None, -1, -1
        
    elif min_length < 256:
        min_length = 128
    
    elif min_length < 512:
        min_length = 256
    
    else:
        min_length = 512
        
    ystart = yaverage - min_length // 2
    yend = yaverage + min_length // 2
    
    xstart = xaverage - min_length // 2
    xend = xaverage + min_length // 2
    
    return img[ystart:ystart + min_length, xstart:xstart + min_length], xstart, ystart
    

In [ ]:
def move_channels_to_front(img, num_channels):
    new_img = img[:,:,0][np.newaxis, :, :]
    for i in range(1,num_channels):
        new_img = np.concatenate((new_img, img[:,:,i][np.newaxis, :, :]), axis=0)
    
    return np.array(new_img)

def move_channels_to_back(img, num_channels):
    new_img = img[0,:,:][:, :, np.newaxis]
    for i in range(1, num_channels):
        new_img = np.concatenate((new_img, img[i, :, :][:, :, np.newaxis]), axis=-1)
    
    return np.array(new_img)

In [ ]:
def reshape_to_512(img):
    if img.shape[0] == 512:
        return img
    resize_scalar = 512 //img.shape[0]
    ret_img = None
    num_channels = 3
    
    if len(img.shape) == 2:
        img = img[:, :, np.newaxis]
        num_channels = 1
    
    with torch.no_grad():

        img_tensor = torch.from_numpy(move_channels_to_front(img, img.shape[-1]))[None, :, :, :].to(torch.float32).to(device)

        up_conv_weight = torch.ones((resize_scalar,resize_scalar))[None,None,:].to(torch.float32).to(device)
        if num_channels == 3:
            up_conv_weight = nn.Parameter(torch.cat((up_conv_weight,up_conv_weight,up_conv_weight), dim=0))

        ret_img = F.conv_transpose2d(img_tensor, up_conv_weight, stride=(resize_scalar, resize_scalar), groups=num_channels)
        ret_img = move_channels_to_back(ret_img[0], ret_img.shape[1]).astype("uint8")
    
    if num_channels == 3:
        return ret_img
    
    return ret_img[:, :, 0]

# recording unique colors and their label in color_dict_new.json and color_dict_new_diseased.json

In [ ]:
def put_in_new_data_dictionary(color_images):
    labeled_colors = {}
    
    for i, img in enumerate(color_images):
        print(".", end="")
        unrolled = np.stack((img[:, :, 0].ravel(), img[:, :, 1].ravel(), img[:, :, 2].ravel()), axis=1)
        index_gray = np.logical_and(np.logical_and(unrolled[:, 0] == unrolled[:, 1], unrolled[:, 1] == unrolled[:, 2]), unrolled[:, 0] == unrolled[:, 2])
        unrolled[index_gray] = (0,0,0)
        unrolled = np.reshape(np.sum(unrolled, axis=-1), (img.shape[0], img.shape[1]))

        labeled_lines = np.full(unrolled.shape, 0)

        x_slices = [unrolled[:, i] for i in range(img.shape[1])]
        complete_xs = []

        for i in range(len(x_slices)):
            count_up = [index for index in range(len(x_slices[i]) - 1) if (x_slices[i][index] == 0 and x_slices[i][index + 1] != 0) or (x_slices[i][index] != 0 and x_slices[i][index + 1] == 0)]
            if len(count_up) == 22:
                for j in range(0, len(count_up), 2):
                    ystart = count_up[j] + 1
                    yend = count_up[j+1] + 1
                    valid_colors = img[ystart:yend, i]
                    valid_colors = ["%s"%((pt[0] << 16) | (pt[1] << 8) | pt[2]) for pt in valid_colors]
                    corr_labels = [(j // 2 + 1)] * len(valid_colors)

                    for key, value in zip(valid_colors, corr_labels):
                        labeled_colors[key] = value

    print("\nnew length of dictionary: %5d"%len(labeled_colors))
    
    with open('color_dict_new.json', 'w') as save_file:
        save_file.write(json.dumps(labeled_colors))

# print(len(color_images))
# put_in_new_data_dictionary(color_images)

# process labeled segmentations and turn them into ground truths

In [ ]:
def label_lines(color_images):

    color_images_labeled = []
    
    with open('color_dict_new.json', 'r') as save_file:
        cDict = json.load(save_file)
        print(len(cDict))
        for i, img in enumerate(color_images):
            print(".", end="")
            unrolled = np.stack((img[:, :, 0].ravel(), img[:, :, 1].ravel(), img[:, :, 2].ravel()), axis=1).astype("int32")
            index_gray = np.logical_and(np.logical_and(unrolled[:, 0] == unrolled[:, 1], unrolled[:, 1] == unrolled[:, 2]), unrolled[:, 0] == unrolled[:, 2])
            index_color = np.logical_not(index_gray)
            unrolled[index_gray] = (0,0,0)
            # encoding color tuple to integer using bitwise operators
            # (255, 170, 35) -> (255 << 16) | (170 << 8) | (35 << 0)
            unrolled = np.bitwise_or(np.bitwise_or(np.left_shift(unrolled[:, 0], 16), np.left_shift(unrolled[:, 1], 8)), unrolled[:, 2]) 

            labeled_lines = np.full(unrolled.shape, 0)
            keys = cDict.keys()

            labeled_lines[index_color] = [cDict["%s"%pt] if "%s"%pt in cDict.keys() else 0 for pt in unrolled[index_color]]

            labeled_lines = np.reshape(labeled_lines, (img.shape[0], img.shape[1]))
            color_images_labeled.append(labeled_lines)
        
        print()
    
    return color_images_labeled
#         fig, arr = plt.subplots(1,1,figsize=(14,10))
#         arr.imshow(labeled_lines)
#         fig.show()
        
        

In [ ]:
def row_col_distinct_bool_array(raveled_transposed_boolean_arr, original_img_shape):
    pts = np.array(np.where(raveled_transposed_boolean_arr)[0])
    col = pts // original_img_shape[1]
    row = pts % original_img_shape[1]
    
    num_distinct = [0] * (img.shape[1]+1)
    prev = 0
    for elem in col:
        prev += 1
        num_distinct[elem+1] = prev
    
    return row, col, num_distinct

def get_index(raveled_transposed_img, label, original_img_shape):
    return row_col_distinct_bool_array(raveled_transposed_img == label, original_img_shape)

In [ ]:
def fill(img, baseline=False):
    
    raveled_transposed_img = img.T.ravel()
    
    if (baseline):
        
        row_end, col_end, num_distinct_end = row_col_distinct_bool_array(raveled_transposed_img != 0, img.shape)
        
        for col in range(img.shape[1]):
            prev_color = 1
            start = False
            color_end = row_end[num_distinct_end[col+1] - 1]
            for row in range(color_end):
                if img[row, col] == 0:
                    if start:
                        img[row, col] = prev_color
                else:
                    if img[row, col] > prev_color:
                        prev_color = img[row, col]
                    
                    if not start:
                        prev_color = img[row, col]
                    
                    start = True
            
            img[color_end:, col] = 12
    else:
        for label in range(1,11):
            row, col, num_distinct = get_index(raveled_transposed_img, label, (512,512))
            row_next, col_next, num_distinct_next = get_index(raveled_transposed_img, label+1, (512,512))

            for i in range(img.shape[1]):
                if num_distinct[i+1] == 0 or num_distinct_next[i+1] == 0:
                    continue
                start = num_distinct[i]
                end = num_distinct_next[i+1] - 1
                img[row[start]:row_next[end], i] = label


#     fig, arr = plt.subplots(1,1,figsize=(14,10))
#     arr.imshow(img)
#     fig.show()
    
    return img


In [ ]:
def only_holes(img):
    col, row = np.where(img.T == 0)
    col = list(set(col))
    if len(col) == 0:
        return None, -1
    return img[:, col], col

In [ ]:
def post_process(img, num_line_fixes=2):
    index_color = np.array(np.where(img.ravel() != 0)[0])
    row = index_color // img.shape[1]
    row_min = row[0]
    row_max = row[-1]
    
    row_11, col_11, num_distinct_11 = get_index(img.T.ravel(), 11, img.shape)
    img[row_11, col_11] = 10
    
    
    
    for i in range(num_line_fixes):
        for row in range(row_min, row_max+1):
            for col in range(1, img.shape[1] - 1):
                if img[row, col-1] == img[row, col+1] and img[row, col] != img[row, col-1]:
                    img[row, col] = img[row, col-1]

                n = 2
                if col > (n-1) and col < img.shape[1] - n and img[row, col-n] == img[row, col+n] and img[row, col] != img[row, col-n]:
                    img[row, col] = img[row, col-n]


                n = 4
                if col > (n-1) and col < img.shape[1] - n and img[row, col-n] == img[row, col+n] and img[row, col] != img[row, col-n]:
                    img[row, col] = img[row, col-n]
    
    unique_labels = np.sort(np.unique(img)[1:])
    row_color, col_color, num_distinct_color = row_col_distinct_bool_array(img.T.ravel() != 0, img.shape)
    
    bg_top_label = 11
    if unique_labels[0] != 1:
        bg_top_label = unique_labels[0]
    
    bg_bot_label = 12 
    if unique_labels[-1] != 10:
        bg_bot_label = unique_labels[-1]
        
    for i in range(img.shape[1]):
        img[:row_color[num_distinct_color[i]], i] = bg_top_label
        img[row_color[num_distinct_color[i+1] - 1]+1:, i] = bg_bot_label
            
#     fig,arr = plt.subplots(1,1,figsize=(14,10))
#     arr.imshow(img)
#     fig.show()
    
    return img


In [ ]:
# './HeidelbergTraining/images_input_labeled/images_input_labeled0299.png'

# loop to create labeled data and corresponding gt label using earlier functions

In [ ]:
# {1920, 1175}
# {1017, 986, 635}
i = 0
    
count = 0
file_count = 0
full_path = "/data/yue/heidelberg_segmentation"
# with open(os.path.join(total_path, "HeidelbergTraining/original_image_paths.txt"), "r") as img_txt_file:
#     with open(os.path.join(total_path, "HeidelbergTraining/original_mask_paths.txt"), "r") as mask_txt_file:

color_images = []
gray_images = []
actual_location = []

total_imgs = 0


for i, (img_path, mask_path) in enumerate(zip(image_path_list, mask_path_list)):
    if "NORMAL" not in img_path:
        continue
    else:
        count += 1
    
    
#     if count < 7:
#         continue
    
#     if count > 3:
#         break
        
#     print(img_path[img_path.find("./"):])
#     print(mask_path[mask_path.find("./"):])
    img1, _, _ = get_black_white(np.array(Image.open(img_path[img_path.find("./"):]).convert("L")))
    img2, crop_start, crop_end = get_black_white(np.array(Image.open(mask_path[mask_path.find("./"):]).convert("L")))

    
    
    color_img2 = np.array(Image.open(mask_path[mask_path.find("./"):]))[50:-100, crop_start:crop_end, :]

#     print(color_img2.shape)
#     print(img2.shape)

    size1 = img1.shape
    size2 = img2.shape

    if size1[0] < 128 or size1[1] < 128:
        continue

    img2 = np.array(Image.fromarray(img2).resize((size1[1], size1[0])))
    color_img2 = np.array(Image.fromarray(color_img2).resize((size1[1], size1[0])))

    translation = match_to_img(img1, color_img2[:, :, 0])
    color_img2 = np.concatenate((color_img2[translation:, :, :], color_img2[:translation, :, :]), axis=0)
    num_image_crops = 0

    cropped_img1, xstart, ystart = get_img_cropped(img1)
    if cropped_img1 is not None:
        total_imgs += 1
        cropped_color_img2 = color_img2[ystart:ystart+cropped_img1.shape[0], xstart:xstart+cropped_img1.shape[1], :]
        cropped_color_img2 = reshape_to_512(cropped_color_img2)
        cropped_img1 = reshape_to_512(cropped_img1)
        color_images.append(cropped_color_img2)
        gray_images.append(cropped_img1)
        actual_location.append((img_path, mask_path))
#         fig,arr = plt.subplots(1, 2, figsize=(14,10))
#         arr[0].imshow(cropped_img1)
#         arr[1].imshow(cropped_color_img2)
#         fig.show()
        num_image_crops += 1

        cropped_img1_half, x2, ystart = get_img_cropped(img1[:, :xstart + cropped_img1.shape[1] // 2])
        if cropped_img1_half is not None:
            cropped_color_img2 = color_img2[ystart:ystart+cropped_img1_half.shape[0], x2:x2+cropped_img1_half.shape[1], :]
            cropped_color_img2 = reshape_to_512(cropped_color_img2)
            cropped_img1_half = reshape_to_512(cropped_img1_half)
            color_images.append(cropped_color_img2)
            gray_images.append(cropped_img1_half)
            actual_location.append((img_path, mask_path))
#             fig,arr = plt.subplots(1, 2, figsize=(14,10))
#             arr[0].imshow(cropped_img1_half)
#             arr[1].imshow(cropped_color_img2)
#             fig.show()
            num_image_crops += 1
        

        cropped_img1_half, x2, ystart = get_img_cropped(img1[:, xstart+cropped_img1.shape[1] // 2:])
        if cropped_img1_half is not None:
            xindex = xstart + cropped_img1.shape[1] // 2 + x2
            cropped_color_img2 = color_img2[ystart:ystart+cropped_img1_half.shape[0], xindex:xindex+cropped_img1_half.shape[1], :]
            cropped_color_img2 = reshape_to_512(cropped_color_img2)
            cropped_img1_half = reshape_to_512(cropped_img1_half)
            color_images.append(cropped_color_img2)
            gray_images.append(cropped_img1_half)
            actual_location.append((img_path, mask_path))
#             fig,arr = plt.subplots(1, 2, figsize=(14,10))
#             arr[0].imshow(cropped_img1_half)
#             arr[1].imshow(cropped_color_img2)
#             fig.show()
            num_image_crops += 1

    print("%s, "%num_image_crops, end="")

    

In [ ]:
print(total_imgs)

In [ ]:
print(len(color_images))
print(len(gray_images))
print(len(actual_location))

In [ ]:
color_images_labeled = label_lines(color_images)
print(len(color_images_labeled))

In [ ]:
for i in range(len(color_images_labeled)): 
    print(".", end="")
    img = np.copy(color_images_labeled[i])
    img = fill(img)
    img = post_process(img)
    
    color_images_labeled[i] = img
    
    img_fp = actual_location[i][0]
    mask_fp = actual_location[i][1]
    
    img_fp = img_fp[img_fp.find("./"):]
    mask_fp = mask_fp[mask_fp.find("./"):]
    
    actual_location[i] = (img_fp, mask_fp)
    
#     fig,arr = plt.subplots(1,2,figsize=(14,10))
#     arr[0].imshow(color_images_labeled[i])
#     arr[1].imshow(img)
#     fig.show()
print()

In [ ]:
labeled_img_save_path = "./HeidelbergTraining/images_input_labeled"
labeled_mask_save_path = "./HeidelbergTraining/images_masks"

print("%s\n%s"%(os.path.isdir(labeled_img_save_path), os.path.isdir(labeled_mask_save_path)))

# num_diseased = 0
num_diseased = 292

with open("./HeidelbergTraining/images_input_labeled_paths.txt", "a") as labeled_file:
    with open("./HeidelbergTraining/images_masks_paths.txt", "a") as mask_file:
        
        img_data = ""
        mask_data = ""
        
        for i,(img, mask, location) in enumerate(zip(gray_images, color_images_labeled, actual_location)):

            img = Image.fromarray(img.astype("uint8"))
            mask = Image.fromarray(mask.astype("uint8"))
            
            img_name = "images_input_labeled%04d"%(i+num_diseased)
            mask_name = "images_masks%04d"%(i+num_diseased)
            
            img.save(os.path.join(labeled_img_save_path, "%s.png"%img_name))
            mask.save(os.path.join(labeled_mask_save_path, "%s.png"%mask_name))
            
            img_data += "%s: %s\n"%(img_name, os.path.abspath(location[0]))
            mask_data += "%s: %s\n"%(mask_name, os.path.abspath(location[1]))
            
            print(".", end="")
        
        labeled_file.write(img_data)
        mask_file.write(mask_data)
print()
            

# helper function to get valid mactel files from pre-created list of valid mactel files

# only diseased images with a mactel severity >= 4 are used as unlabeled data for semi-supervised learning

## severities can be found here: /data/yue/mactel/representation/mactel_gradings_OPT_OCT_Volume IR_combined_closest_6m.csv

In [ ]:
def get_dir_list(path="", lower_bound=0, upper_bound=10000, dot_num=10000):
    if not os.path.isdir(path):
        print("not a valid path")
        return []
    
    f = os.scandir(path)
    
    files = []
    i = -1
    for entry in f:
        i += 1
        if i < lower_bound:
            continue
            
        elif i % dot_num == 0 and i > 0:
            print(".", end=" ")
            
        if i >= upper_bound:
            break
            
        if entry.is_file():
            files.append(entry.path)
    
    print("." if upper_bound % dot_num > 0 else "")
    print("files from [%s, %s)"%(lower_bound, upper_bound))
    print("each dot above represents ~%s files scanned (len(files)/%s + 1)\nnum files in list: %s"%(dot_num, dot_num, len(files)))
    return files
    

In [ ]:
def get_valid_mactel_file(mactel_gradings, full_name="", verbose=False):
    d_id = full_name[full_name.find("converted/")+10:full_name.find(".1_")+2]
    if verbose:
        print(d_id)
    row = mactel_gradings[mactel_gradings["dicomID"] == d_id]

    if len(row) > 0:
        index = row.iloc[0].name
        valid = int(mactel_gradings[index:index+1]["stage"])
        if verbose:
            print("matching dicom ID found!\ndicom ID: %s\nmactel severity: %s"%(row["dicomID"][index], valid))
            if valid < 4:
                print()
        return valid

    else:
        if verbose:
            print("no matching dicom ID found in CSV\n")
        return -1

# loop to create unlabeled dataset

In [ ]:
path = "../mactel/representation/mactel_gradings_OPT_OCT_Volume IR_combined_closest_6m.csv"
mactel_gradings = pd.read_csv(path, delimiter=",")
# print(mactel_gradings[:1])

numfiles = get_dir_list(path="../mactel/representation/dicom_converted", lower_bound=0, upper_bound=100000)
print(len(numfiles))

In [ ]:
i = -1
count = 0

print(len(numfiles))

severity5 = []
severity4 = []

for file in numfiles:
    i += 1
    
    file_id = get_valid_mactel_file(mactel_gradings, full_name=file, verbose=False)
    if (file_id > 3):
#         print("hooray\n")
        count += 1
        if (file_id % 10 == 5):
            severity5.append(file)
        else:
            severity4.append(file)


print("num file w/ mactel severity >= 4: %s\nnum files with 4: %s\nnumfiles with 5: %s"%(count, len(severity4), len(severity5)))
      

In [ ]:
print(os.path.isdir("./HeidelbergTraining/images_input_unlabeled"))

In [ ]:
unlabeled_img_path_list = severity5

count = 0
unlabeled_img_save_path = "./HeidelbergTraining/images_input_unlabeled"


def crop_and_save_unlabeled_images(unlabeled_img_path_list, unlabeled_img_save_path, count):
    for i, unlabeled_img_path in enumerate(sorted(unlabeled_img_path_list)):
        if i % 50 == 0:
            print(".", end="")

        unlabeled_img = np.array(Image.open(unlabeled_img_path))
        with open("./HeidelbergTraining/images_input_unlabeled_paths.txt", "a") as unlabeled_save_file:
            cropped_img1, xstart, ystart = get_img_cropped(unlabeled_img)
            if cropped_img1 is not None:
                save_img = Image.fromarray(reshape_to_512(cropped_img1).astype("uint8"))
                name = "images_input_unlabeled%04d.png"%count
                save_img.save(os.path.join(unlabeled_img_save_path, name))
                unlabeled_save_file.write("%s: %s\n"%(name, os.path.abspath(unlabeled_img_path)))
                count += 1

#                 plt.imshow(save_img)
#                 plt.show()

                cropped_img1_half, x2, ystart = get_img_cropped(unlabeled_img[:, :xstart + cropped_img1.shape[1] // 2])
                if cropped_img1_half is not None:
                    save_img = Image.fromarray(reshape_to_512(cropped_img1_half).astype("uint8"))
                    name = "images_input_unlabeled%04d.png"%count
                    save_img.save(os.path.join(unlabeled_img_save_path, name))
                    unlabeled_save_file.write("%s: %s\n"%(name, os.path.abspath(unlabeled_img_path)))
                    count += 1

#                     plt.imshow(save_img)
#                     plt.show()

                cropped_img1_half, x2, ystart = get_img_cropped(unlabeled_img[:, xstart+cropped_img1.shape[1] // 2:])
                if cropped_img1_half is not None:
                    save_img = Image.fromarray(reshape_to_512(cropped_img1_half).astype("uint8"))
                    name = "images_input_unlabeled%04d.png"%count
                    save_img.save(os.path.join(unlabeled_img_save_path, name))
                    unlabeled_save_file.write("%s: %s\n"%(name, os.path.abspath(unlabeled_img_path)))
                    count += 1

#                     plt.imshow(save_img)
#                     plt.show()
    print()
    return count 
                

In [ ]:
with open("./HeidelbergTraining/images_input_unlabeled_paths.txt", "w") as file:
        print("cleared previous data in images_input_unlabeled_paths.txt")
num_files = crop_and_save_unlabeled_images(severity5, "./HeidelbergTraining/images_input_unlabeled", 0)
num_files = crop_and_save_unlabeled_images(severity4, "./HeidelbergTraining/images_input_unlabeled", num_files)
print(num_files)

# loop to create baseline dataset

In [ ]:
baseline_p = "./data/Segmentations diseased eyes - MacTel/MacTel eyes with segmentations"
labeled_paths = []
baseline_paths = []
num_diseased = 292
diseased_train_cutoff = 230
diseased_valid_cutoff = 262

actual_locations = []

with open("./HeidelbergTraining/images_input_labeled_paths.txt", "r") as labeled_file:
    
    labeled_paths = sorted(labeled_file.readlines())[diseased_valid_cutoff:num_diseased]
    actual_locations = ["images_masks%s"%elem[elem.find(":") - 4:elem.find(":")].strip() for elem in labeled_paths]
    labeled_paths = [elem[elem.find(":")+2:].strip() for elem in labeled_paths]
    
    baseline_paths = [elem[elem.find("NHOR"):elem.find("..png")] for elem in labeled_paths]
    baseline_paths = [os.path.join(baseline_p, "%s.png"%elem) for elem in baseline_paths]



In [ ]:
print(actual_locations)

In [ ]:
mask_paths = []
m_path = "./HeidelbergTraining/images_masks"
for dir_name, dirs, files in os.walk(m_path):
    mask_paths = sorted([os.path.join(m_path, f) for f in files if f.endswith(".png")])

mask_paths = mask_paths[diseased_valid_cutoff:num_diseased]

In [ ]:
labeled_paths = sorted(list(set(labeled_paths)))
baseline_paths = sorted(list(set(baseline_paths)))


In [ ]:
corresponding_paths = []

In [ ]:
count = 0
baseline_image_list = []

for i in range(len(baseline_paths)):
    baseline_p = baseline_paths[i]
    labeled_p = labeled_paths[i]
    
    img_labeled, crop_start, crop_end = get_black_white(np.array(Image.open(labeled_p).convert("L")))
    img_baseline, _, _ = get_black_white(np.array(Image.open(baseline_p).convert("L")))
    
    baseline_img_color = np.array(Image.open(baseline_p))[50:-100, crop_start:crop_end]
    baseline_img_color = np.array(Image.fromarray(baseline_img_color.astype("uint8")).resize((img_labeled.shape[1], img_labeled.shape[0])))
    
    translation = match_to_img(img_labeled, baseline_img_color[:, :, 0])
    baseline_img_color = np.concatenate((baseline_img_color[translation:, :, :], baseline_img_color[:translation, :, :]), axis=0)
    
    cropped_img1, xstart, ystart = get_img_cropped(img_labeled)
    if cropped_img1 is not None:
        cropped_color_img2 = baseline_img_color[ystart:ystart+cropped_img1.shape[0], xstart:xstart+cropped_img1.shape[1], :]
        cropped_color_img2 = reshape_to_512(cropped_color_img2)
        cropped_img1 = reshape_to_512(cropped_img1)
#         fig,arr = plt.subplots(1, 3, figsize=(14,10))
#         arr[0].imshow(cropped_img1)
#         arr[2].imshow(np.array(Image.open(mask_paths[count]).convert("L")))
#         arr[1].imshow(cropped_color_img2)
#         fig.show()
#         plt.show()
    
        baseline_image_list.append(cropped_color_img2)
        count += 1

        
        
        cropped_img1_half, x2, ystart = get_img_cropped(img_labeled[:, :xstart + cropped_img1.shape[1] // 2])
        if cropped_img1_half is not None:
            cropped_color_img2 = baseline_img_color[ystart:ystart+cropped_img1_half.shape[0], x2:x2+cropped_img1_half.shape[1], :]
            cropped_color_img2 = reshape_to_512(cropped_color_img2)
            cropped_img1_half = reshape_to_512(cropped_img1_half)
#             fig,arr = plt.subplots(1, 3, figsize=(14,10))
#             arr[0].imshow(cropped_img1_half)
#             arr[2].imshow(np.array(Image.open(mask_paths[count]).convert("L")))
#             arr[1].imshow(cropped_color_img2)
#             fig.show()
#             plt.show()
            
            baseline_image_list.append(cropped_color_img2)
            count += 1
        

        cropped_img1_half, x2, ystart = get_img_cropped(img_labeled[:, xstart+cropped_img1.shape[1] // 2:])
        if cropped_img1_half is not None:
            xindex = xstart + cropped_img1.shape[1] // 2 + x2
            cropped_color_img2 = baseline_img_color[ystart:ystart+cropped_img1_half.shape[0], xindex:xindex+cropped_img1_half.shape[1], :]
            cropped_color_img2 = reshape_to_512(cropped_color_img2)
            cropped_img1_half = reshape_to_512(cropped_img1_half)
#             fig,arr = plt.subplots(1, 3, figsize=(14,10))
#             arr[0].imshow(cropped_img1_half)
#             arr[2].imshow(np.array(Image.open(mask_paths[count]).convert("L")))
#             arr[1].imshow(cropped_color_img2)
#             fig.show()
#             plt.show()
            
            baseline_image_list.append(cropped_color_img2)
            count += 1
    
    print(baseline_img_color.shape)
    print(img_labeled.shape)
    
    

In [ ]:
baseline_image_list_labeled = label_lines(baseline_image_list)

In [ ]:
print(len(baseline_image_list_labeled))
print(len(actual_locations))

In [ ]:
N=23

fig,arr = plt.subplots(1,2,figsize=(14,10))
arr[0].imshow(baseline_image_list_labeled[N])
arr[1].imshow(np.array(Image.open("./HeidelbergTraining/images_masks/%s.png"%actual_locations[N]).convert("L")))
fig.show()
plt.show()



In [ ]:
corresponding_locations = []

In [ ]:
full_image_save_path = "./HeidelbergTraining/images_baseline/baseline_full_image"
print(os.path.isdir(full_image_save_path))

In [ ]:
with open("./HeidelbergTraining/images_baseline_paths.txt", "w") as baseline_save_file:
    print("cleared")

mask_save_path = "./HeidelbergTraining/images_baseline/baseline_masks"
image_save_path = "./HeidelbergTraining/images_baseline/baseline_images"
full_image_save_path = "./HeidelbergTraining/images_baseline/baseline_full_image"

count = 0
    
for i in range(len(baseline_image_list_labeled)):
#     if i > 3:
#         break
#     print(".", end="")
    print(i)
    img = np.copy(baseline_image_list_labeled[i])
    img = fill(img, baseline=True)
    img = post_process(img)
    mask = np.array(Image.open(mask_paths[i]).convert("L"))
    
    mask, col = only_holes(mask)
    if mask is not None:
        full_img = Image.fromarray(img.astype("uint8"))
        full_img.save(os.path.join(full_image_save_path, "baseline_full_image%04d.png"%i))
        
        img = img[:, col]
#         print(mask.shape)
#         fig,arr = plt.subplots(1,2,figsize=(14,10))
#         arr[0].imshow(img)
#         arr[1].imshow(mask)
#         fig.show()
#         plt.show()

        img = Image.fromarray(img.astype("uint8"))
        mask = Image.fromarray(mask.astype("uint8"))
        
        with open("./HeidelbergTraining/images_baseline_paths.txt", "a") as baseline_save_file:
            img_name = "baseline_images%04d"%i
            mask_name = "baseline_masks%04d"%i
            
            img.save(os.path.join(image_save_path, "%s.png"%img_name))
            mask.save(os.path.join(mask_save_path, "%s.png"%mask_name))
            
            baseline_save_file.write("%s, %s: %s\n"%(img_name,mask_name,"./HeidelbergTraining/images_masks/%s.png"%actual_locations[count]))
            
    count += 1
    